In [1]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def nhl_calendar(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates={}".format(season)
    resp = download(url=url)
    if resp is not None:
        txt = json.loads(resp)['leagues'][0]['calendar']
        datenum = list(map(lambda x: x[:10].replace("-",""),txt))
        date = list(map(lambda x: x[:10],txt))

        year = list(map(lambda x: x[:4],txt))
        month = list(map(lambda x: x[5:7],txt))
        day = list(map(lambda x: x[8:10],txt))

        data = {"season": season,
                "datetime" : txt,
                "date" : date,
                "year": year,
                "month": month,
                "day": day,
                "dateURL": datenum

               }
        df = pd.DataFrame(data)
        df['url']="http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates="
        df['url']= df['url'] + df['dateURL']
        return df
    else:
        pass

nhl_calendar(2018)

,season,datetime,date,year,month,day,dateURL,url
0,2018,2017-09-16T07:00Z,2017-09-16,2017,09,16,20170916,http://site.api.espn.com/apis/site/v2/sports/h...
1,2018,2017-09-17T07:00Z,2017-09-17,2017,09,17,20170917,http://site.api.espn.com/apis/site/v2/sports/h...
2,2018,2017-09-18T07:00Z,2017-09-18,2017,09,18,20170918,http://site.api.espn.com/apis/site/v2/sports/h...
3,2018,2017-09-19T07:00Z,2017-09-19,2017,09,19,20170919,http://site.api.espn.com/apis/site/v2/sports/h...
4,2018,2017-09-20T07:00Z,2017-09-20,2017,09,20,20170920,http://site.api.espn.com/apis/site/v2/sports/h...
...,...,...,...,...,...,...,...,...
235,2018,2018-05-28T07:00Z,2018-05-28,2018,05,28,20180528,http://site.api.espn.com/apis/site/v2/sports/h...
236,2018,2018-05-30T07:00Z,2018-05-30,2018,05,30,20180530,http://site.api.espn.com/apis/site/v2/sports/h...
237,2018,2018-06-02T07:00Z,2018-06-02,2018,06,02,20180602,http://site.api.espn.com/apis/site/v2/sports/h...
238,2018,2018-06-04T07:00Z,2018-06-04,2018,06,04,20180604,http://site.api.espn.com/apis/site/v2/sports/h...


In [2]:
pd.set_option('display.max_colwidth',None)
schedule = {}
for x in range(2001,2022):
    full_schedule = nhl_calendar(x)
    if full_schedule is not None:
        full_schedule.to_json(orient='records')
        schedule[x]=full_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('nhl_calendar.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)    

Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=2004
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=2004
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=2004
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=2004
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=2004
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=2004


In [3]:
def nhl_schedule(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates={}".format(season)
    resp = download(url=url)
    ev = pd.DataFrame()
    if resp is not None:
        txt = json.loads(resp)['leagues'][0]['calendar']
    #     print(len(txt))
        txt = list(map(lambda x: x[:10].replace("-",""),txt))
        i=0
        for date in txt:
            print(f"Working on {season}: {i+1} of {len(txt)}, date: {txt[i]}")
            url = "http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates={}&limit=1000".format(date)
            resp = download(url=url)

            if resp is not None:
                events_txt = json.loads(resp)

                events = events_txt['events']
                for event in events:
                    bad_keys = ['linescores', 'statistics', 'leaders',  'records']
                    for k in bad_keys:
                        if k in event['competitions'][0]['competitors'][0].keys():
                            del event['competitions'][0]['competitors'][0][k]
                        if k in event['competitions'][0]['competitors'][1].keys():
                            del event['competitions'][0]['competitors'][1][k]
                    if 'links' in event['competitions'][0]['competitors'][0]['team'].keys():
                        del event['competitions'][0]['competitors'][0]['team']['links']
                    if 'links' in event['competitions'][0]['competitors'][1]['team'].keys():
                        del event['competitions'][0]['competitors'][1]['team']['links']    
                    if event['competitions'][0]['competitors'][0]['homeAway']=='home':
                        event['competitions'][0]['home'] = event['competitions'][0]['competitors'][0]['team']    
                    else: 
                        event['competitions'][0]['away'] = event['competitions'][0]['competitors'][0]['team']
                    if event['competitions'][0]['competitors'][1]['homeAway']=='away':
                        event['competitions'][0]['away'] = event['competitions'][0]['competitors'][1]['team']
                    else: 
                        event['competitions'][0]['home'] = event['competitions'][0]['competitors'][1]['team']

                    del_keys = ['competitors', 'broadcasts','geoBroadcasts', 'headlines']
                    for k in del_keys:
                        if k in event['competitions'][0].keys():
                            del event['competitions'][0][k]

                    ev = ev.append(pd.json_normalize(event['competitions'][0]))
                i+=1
                ev['season']=season
            else:
                i+=1
    return ev


In [4]:
schedule_table = pd.DataFrame()
schedule = {}
for x in range(2002,2022):
    year_schedule = nhl_schedule(x)
    if year_schedule is not None:
        year_schedule.to_csv(f"nhl/schedules/nhl_games_info_{x}.csv")
        schedule_table = schedule_table.append(year_schedule)
        year_schedule.to_json(orient='records')
        schedule[x]=year_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}



schedule_table.to_csv('nhl_games_info_2002_2021.csv')
    
# event['competitions'][0]

Working on 2002: 1 of 221, date: 20011003
Working on 2002: 2 of 221, date: 20011004
Working on 2002: 3 of 221, date: 20011005
Working on 2002: 4 of 221, date: 20011006
Working on 2002: 5 of 221, date: 20011007
Working on 2002: 6 of 221, date: 20011008
Working on 2002: 7 of 221, date: 20011009
Working on 2002: 8 of 221, date: 20011010
Working on 2002: 9 of 221, date: 20011011
Working on 2002: 10 of 221, date: 20011012
Working on 2002: 11 of 221, date: 20011013
Working on 2002: 12 of 221, date: 20011014
Working on 2002: 13 of 221, date: 20011015
Working on 2002: 14 of 221, date: 20011016
Working on 2002: 15 of 221, date: 20011017
Working on 2002: 16 of 221, date: 20011018
Working on 2002: 17 of 221, date: 20011019
Working on 2002: 18 of 221, date: 20011020
Working on 2002: 19 of 221, date: 20011021
Working on 2002: 20 of 221, date: 20011022
Working on 2002: 21 of 221, date: 20011023
Working on 2002: 22 of 221, date: 20011024
Working on 2002: 23 of 221, date: 20011025
Working on 2002: 24 

Working on 2002: 157 of 221, date: 20020327
Working on 2002: 158 of 221, date: 20020328
Working on 2002: 159 of 221, date: 20020329
Working on 2002: 160 of 221, date: 20020330
Working on 2002: 161 of 221, date: 20020331
Working on 2002: 162 of 221, date: 20020401
Working on 2002: 163 of 221, date: 20020402
Working on 2002: 164 of 221, date: 20020403
Working on 2002: 165 of 221, date: 20020404
Working on 2002: 166 of 221, date: 20020405
Working on 2002: 167 of 221, date: 20020406
Working on 2002: 168 of 221, date: 20020407
Working on 2002: 169 of 221, date: 20020408
Working on 2002: 170 of 221, date: 20020409
Working on 2002: 171 of 221, date: 20020410
Working on 2002: 172 of 221, date: 20020411
Working on 2002: 173 of 221, date: 20020412
Working on 2002: 174 of 221, date: 20020413
Working on 2002: 175 of 221, date: 20020414
Working on 2002: 176 of 221, date: 20020417
Working on 2002: 177 of 221, date: 20020418
Working on 2002: 178 of 221, date: 20020419
Working on 2002: 179 of 221, dat

Working on 2003: 125 of 236, date: 20030126
Working on 2003: 126 of 236, date: 20030127
Working on 2003: 127 of 236, date: 20030128
Working on 2003: 128 of 236, date: 20030129
Working on 2003: 129 of 236, date: 20030130
Working on 2003: 130 of 236, date: 20030202
Working on 2003: 131 of 236, date: 20030204
Working on 2003: 132 of 236, date: 20030205
Working on 2003: 133 of 236, date: 20030206
Working on 2003: 134 of 236, date: 20030207
Working on 2003: 135 of 236, date: 20030208
Working on 2003: 136 of 236, date: 20030209
Working on 2003: 137 of 236, date: 20030210
Working on 2003: 138 of 236, date: 20030211
Working on 2003: 139 of 236, date: 20030212
Working on 2003: 140 of 236, date: 20030213
Working on 2003: 141 of 236, date: 20030214
Working on 2003: 142 of 236, date: 20030215
Working on 2003: 143 of 236, date: 20030216
Working on 2003: 144 of 236, date: 20030217
Working on 2003: 145 of 236, date: 20030218
Working on 2003: 146 of 236, date: 20030219
Working on 2003: 147 of 236, dat

Working on 2005: 63 of 178, date: 20041214
Working on 2005: 64 of 178, date: 20041215
Working on 2005: 65 of 178, date: 20041216
Working on 2005: 66 of 178, date: 20041217
Working on 2005: 67 of 178, date: 20041218
Working on 2005: 68 of 178, date: 20041219
Working on 2005: 69 of 178, date: 20041220
Working on 2005: 70 of 178, date: 20041221
Working on 2005: 71 of 178, date: 20041222
Working on 2005: 72 of 178, date: 20041223
Working on 2005: 73 of 178, date: 20041226
Working on 2005: 74 of 178, date: 20041227
Working on 2005: 75 of 178, date: 20041228
Working on 2005: 76 of 178, date: 20041229
Working on 2005: 77 of 178, date: 20041230
Working on 2005: 78 of 178, date: 20041231
Working on 2005: 79 of 178, date: 20050101
Working on 2005: 80 of 178, date: 20050102
Working on 2005: 81 of 178, date: 20050103
Working on 2005: 82 of 178, date: 20050104
Working on 2005: 83 of 178, date: 20050105
Working on 2005: 84 of 178, date: 20050106
Working on 2005: 85 of 178, date: 20050107
Working on 

Working on 2006: 58 of 239, date: 20051114
Working on 2006: 59 of 239, date: 20051115
Working on 2006: 60 of 239, date: 20051116
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=20051116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=20051116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=20051116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=20051116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=20051116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/scoreboard?dates=20051116&limit=1000
Working on 2006: 61 of 239, date: 20051117
Working on 2006: 62 of 239, date: 20051118
Working on 2006: 63 of 239, date: 20051119
Working on 2006:

...

Working on 2017: 156 of 234, date: 20170308
Working on 2017: 157 of 234, date: 20170309
Working on 2017: 158 of 234, date: 20170310
Working on 2017: 159 of 234, date: 20170311
Working on 2017: 160 of 234, date: 20170312
Working on 2017: 161 of 234, date: 20170313
Working on 2017: 162 of 234, date: 20170314
Working on 2017: 163 of 234, date: 20170315
Working on 2017: 164 of 234, date: 20170316
Working on 2017: 165 of 234, date: 20170317
Working on 2017: 166 of 234, date: 20170318
Working on 2017: 167 of 234, date: 20170319
Working on 2017: 168 of 234, date: 20170320
Working on 2017: 169 of 234, date: 20170321
Working on 2017: 170 of 234, date: 20170322
Working on 2017: 171 of 234, date: 20170323
Working on 2017: 172 of 234, date: 20170324
Working on 2017: 173 of 234, date: 20170325
Working on 2017: 174 of 234, date: 20170326
Working on 2017: 175 of 234, date: 20170327
Working on 2017: 176 of 234, date: 20170328
Working on 2017: 177 of 234, date: 20170329
Working on 2017: 178 of 234, dat

Working on 2018: 111 of 240, date: 20180110
Working on 2018: 112 of 240, date: 20180111
Working on 2018: 113 of 240, date: 20180112
Working on 2018: 114 of 240, date: 20180113
Working on 2018: 115 of 240, date: 20180114
Working on 2018: 116 of 240, date: 20180115
Working on 2018: 117 of 240, date: 20180116
Working on 2018: 118 of 240, date: 20180117
Working on 2018: 119 of 240, date: 20180118
Working on 2018: 120 of 240, date: 20180119
Working on 2018: 121 of 240, date: 20180120
Working on 2018: 122 of 240, date: 20180121
Working on 2018: 123 of 240, date: 20180122
Working on 2018: 124 of 240, date: 20180123
Working on 2018: 125 of 240, date: 20180124
Working on 2018: 126 of 240, date: 20180125
Working on 2018: 127 of 240, date: 20180128
Working on 2018: 128 of 240, date: 20180130
Working on 2018: 129 of 240, date: 20180131
Working on 2018: 130 of 240, date: 20180201
Working on 2018: 131 of 240, date: 20180202
Working on 2018: 132 of 240, date: 20180203
Working on 2018: 133 of 240, dat

Working on 2019: 59 of 241, date: 20181114
Working on 2019: 60 of 241, date: 20181115
Working on 2019: 61 of 241, date: 20181116
Working on 2019: 62 of 241, date: 20181117
Working on 2019: 63 of 241, date: 20181118
Working on 2019: 64 of 241, date: 20181119
Working on 2019: 65 of 241, date: 20181120
Working on 2019: 66 of 241, date: 20181121
Working on 2019: 67 of 241, date: 20181123
Working on 2019: 68 of 241, date: 20181124
Working on 2019: 69 of 241, date: 20181125
Working on 2019: 70 of 241, date: 20181126
Working on 2019: 71 of 241, date: 20181127
Working on 2019: 72 of 241, date: 20181128
Working on 2019: 73 of 241, date: 20181129
Working on 2019: 74 of 241, date: 20181130
Working on 2019: 75 of 241, date: 20181201
Working on 2019: 76 of 241, date: 20181202
Working on 2019: 77 of 241, date: 20181203
Working on 2019: 78 of 241, date: 20181204
Working on 2019: 79 of 241, date: 20181205
Working on 2019: 80 of 241, date: 20181206
Working on 2019: 81 of 241, date: 20181207
Working on 

Working on 2020: 6 of 247, date: 20190920
Working on 2020: 7 of 247, date: 20190921
Working on 2020: 8 of 247, date: 20190922
Working on 2020: 9 of 247, date: 20190923
Working on 2020: 10 of 247, date: 20190924
Working on 2020: 11 of 247, date: 20190925
Working on 2020: 12 of 247, date: 20190926
Working on 2020: 13 of 247, date: 20190927
Working on 2020: 14 of 247, date: 20190928
Working on 2020: 15 of 247, date: 20190929
Working on 2020: 16 of 247, date: 20191002
Working on 2020: 17 of 247, date: 20191003
Working on 2020: 18 of 247, date: 20191004
Working on 2020: 19 of 247, date: 20191005
Working on 2020: 20 of 247, date: 20191006
Working on 2020: 21 of 247, date: 20191007
Working on 2020: 22 of 247, date: 20191008
Working on 2020: 23 of 247, date: 20191009
Working on 2020: 24 of 247, date: 20191010
Working on 2020: 25 of 247, date: 20191011
Working on 2020: 26 of 247, date: 20191012
Working on 2020: 27 of 247, date: 20191013
Working on 2020: 28 of 247, date: 20191014
Working on 2020

Working on 2020: 195 of 247, date: 20200404
Working on 2020: 196 of 247, date: 20200801
Working on 2020: 197 of 247, date: 20200802
Working on 2020: 198 of 247, date: 20200803
Working on 2020: 199 of 247, date: 20200804
Working on 2020: 200 of 247, date: 20200805
Working on 2020: 201 of 247, date: 20200806
Working on 2020: 202 of 247, date: 20200807
Working on 2020: 203 of 247, date: 20200808
Working on 2020: 204 of 247, date: 20200809
Working on 2020: 205 of 247, date: 20200811
Working on 2020: 206 of 247, date: 20200812
Working on 2020: 207 of 247, date: 20200813
Working on 2020: 208 of 247, date: 20200814
Working on 2020: 209 of 247, date: 20200815
Working on 2020: 210 of 247, date: 20200816
Working on 2020: 211 of 247, date: 20200817
Working on 2020: 212 of 247, date: 20200818
Working on 2020: 213 of 247, date: 20200819
Working on 2020: 214 of 247, date: 20200820
Working on 2020: 215 of 247, date: 20200821
Working on 2020: 216 of 247, date: 20200822
Working on 2020: 217 of 247, dat